In [8]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

np.set_printoptions(6, suppress=True)

plt.style.use([
    'grid',
    'notebook'
])

In [9]:
A_vertices = []
B_vertices = []

Ts = .5

r = 31
mu = 4
sigma = 55
A2 = np.pi * r**2

Kb = 13.61200274

In [10]:
h1_interval = np.linspace(20, 74, 1000)
h2_interval = np.linspace(12, 50, 1000)

Z1 = []
Z2 = []
Z3 = []

c = 0
min_diff = 6


def q_34(diff):
        return (33.082144*diff + 89.99671)*0.89930283

def q_out(h4):
    return (9.785984*h4 + 156.704985)*1.18231989


for i in range(len(h1_interval)):
    h1 = h1_interval[i]
    
    for j in range(i, len(h1_interval)): 
        h2 = h2_interval[j]

        diff = h1 - h2

        if diff < min_diff:
            continue
        
        R34 = diff / q_34(diff)
        qout = q_out(h2)

        a3 = (3 * r / 5) * (2.7 * r - ((np.cos(2.5*np.pi*h1 - mu)) / (sigma * np.sqrt(2 * np.pi))) * np.exp(-((h1 - mu)**2) / (2 * sigma ** 2)))

        z1 = 1/R34
        z2 = qout/h2
        z3 = 1/a3

        Z1.append(z1)
        Z2.append(z2)
        Z3.append(z3)


min_z1, max_z1 = min(Z1), max(Z1)
min_z2, max_z2 = min(Z2), max(Z2)
min_z3, max_z3 = min(Z3), max(Z3)

print(min_z1, max_z1)
print(min_z2, max_z2)
print(min_z3, max_z3)

33.12312805473791 43.23991505394907
15.27567193897479 27.0097819120594
0.0006422816947170168 0.0006423877647818338


In [11]:
z1_bounds = np.array([min_z1, max_z1])
z2_bounds = np.array([min_z2, max_z2])
z3_bounds = np.array([min_z3, max_z3])

np.save('./data/z1_bounds.npy', z1_bounds)
np.save('./data/z2_bounds.npy', z2_bounds)
np.save('./data/z3_bounds.npy', z3_bounds)

for z1 in z1_bounds:
    for z2 in z2_bounds:
        for z3 in z3_bounds:
            Ai = np.array([
                [-z1*z3, z1*z3],
                [z1/A2, (-z1-z2)/A2]
            ])

            Bi = np.array([
                [Kb*z3],
                [.0]
            ])

            print(
                np.linalg.eigvals(Ai)
            )

            A_vertices.append(Ai)
            B_vertices.append(Bi)

[-0.034154 -0.003152]
[-0.034157 -0.003152]
[-0.035889 -0.005303]
[-0.035892 -0.005304]
[-0.043957 -0.003197]
[-0.043962 -0.003197]
[-0.045591 -0.00545 ]
[-0.045595 -0.00545 ]


In [12]:
Z1_, Z2_, Z3_ = sp.symbols('Z1 Z2 Z3')

z1_min, z1_max = z1_bounds
z2_min, z2_max = z2_bounds
z3_min, z3_max = z3_bounds

M1 = (z1_max - Z1_) / (z1_max - z1_min)
N1 = (z2_max - Z2_) / (z2_max - z2_min)
P1 = (z3_max - Z3_) / (z3_max - z3_min)

M2 = 1 - M1
N2 = 1 - N1
P2 = 1 - P1

In [13]:
print(' ')
print(M1)
print(N1)
print(P1)
print(' ')
print(M2)
print(N2)
print(P2)

 
4.27407585603222 - 0.0988456117617158*Z1
2.30181768996658 - 0.0852216318317941*Z2
6056.25881242226 - 9427730.63942006*Z3
 
0.0988456117617158*Z1 - 3.27407585603222
0.0852216318317941*Z2 - 1.30181768996658
9427730.63942006*Z3 - 6055.25881242226


In [14]:
for i in range(len(A_vertices)):
    A = A_vertices[i]
    B = B_vertices[i]
    
    np.save(f'./data/vertices/A_{i}', A)
    np.save(f'./data/vertices/B_{i}', B)